In [1]:
#!pip install beautifulsoup4 requests

In [2]:

import requests
from bs4 import BeautifulSoup

In [ ]:

url = 'https://Your-url'
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

r = requests.get("https://www.materialepyramiden.dk/",headers=headers,timeout=10)
soup = BeautifulSoup(r.content)
print(soup)

In [ ]:

'''
with open("C:\\Users\\frans\\OneDrive\\Dev_OneDrive\\Rhino\\GH_LCA\\pytamiden\\index.html",encoding='utf8') as fp:
    soup = BeautifulSoup(fp, "html")
'''

In [ ]:

results = soup.find_all("div", class_="mat-overlay")
links =[]
for i in range(len(results)):
    links.append(results[i].attrs['data-link'])

print(links)
print(len(links))

In [ ]:
# DOWNLOAD ALL MATERIAL FILES

for i in range(len(links)):
    name = links[i].split("=")[-1]
    

    url = links[i]
    print(f"downloading {name} from {url}")
    ua = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    response  = requests.get(url, headers={"User-Agent": ua})
    if response.status_code == 200:
        with open(f"./materialer/{name}.html","wb") as file:
            file.write(response.content)
            file.close()
    else:
        print("Bad response code :", response.status_code, url)
    

print("Done")

In [ ]:
# import OS module

import os
from tkinter.tix import DirList
import sqlite3
 
# Get the list of all files and directories
path = "./materialer/"
dir_list = os.listdir(path)
sqliteConnection = sqlite3.connect('./dataBase/materialDB_20240130.db')

cursor = sqliteConnection.cursor()
tableName = "materials"

query = f"DROP TABLE {tableName}"
#cursor.execute(query)

query = f"""CREATE TABLE IF NOT EXISTS {tableName}(
    name	TEXT NOT NULL UNIQUE,
    category TEXT NOT NULL,
    density NUMERIC NOT NULL,
    GWP NUMERIC NOT NULL,
    ODB NUMERIC NOT NULL,
    POCP NUMERIC NOT NULL,
    EP NUMERIC NOT NULL,
    AP NUMERIC NOT NULL,
	PRIMARY KEY("name")
);"""
cursor.execute(query)


cloumnList = "name,category,density,GWP,ODB,POCP,EP,AP"

#for i in range(len(dir_list)):
runLen = len(dir_list)
#runLen = 1
for i in range(runLen):
    filepath = f"{path}{dir_list[i]}"
    print(f"opening {filepath}")
    with open(filepath, "r",encoding='utf8') as file:
        soup = BeautifulSoup(file, "html")

        name = soup.find( class_ = "single-title single-text c1" ).find(class_ = "en").text
        print(f"found name {name}")

        density = ""
        insulation = ""
        cat = ""
        GWP = ""
        ODP = ""
        POCP  = ""
        EP = ""
        AP = ""



        #table = soup.find('table',class_ = "single-table")
        rows = soup.find_all('tr', class_ = 'tr-line')
        print(len(rows))

        
        for row in rows:
            #print(row)
            #print("   ")
            if(row.find(class_ = 'en')):
                if(row.find(class_ = 'en').text == "Density:"):
                    density = row.find(class_ = 't-text t-number t4').text.split(" ")[0]
            
            if(row.find(class_ = 'en')):
                if(row.find(class_ = 'en').text == "Materialgroup:"):
                    cat = row.find(class_ = 't-text t-number t4').text
            
            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Global Warming Potential [GWP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        GWP = row.find(class_ = 't-text t-number t2').text

            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Ozone Depletion Potential [ODP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        ODP = row.find(class_ = 't-text t-number t2').text

            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Photochemical Ozone Depletion Potential [POCP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        POCP = row.find(class_ = 't-text t-number t2').text

            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Eutrophication Potential [EP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        EP = row.find(class_ = 't-text t-number t2').text
       
            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Acidification Potential [AP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        AP = row.find(class_ = 't-text t-number t2').text



            #print("   ")


        print(f"name:{name} , category:{cat}, density:{density}, GWP:{GWP}, ODP:{ODP},POCP:{POCP},EP:{EP},AP:{AP}" )
        
        query = f"""INSERT OR REPLACE INTO {tableName} ({cloumnList})
                    VALUES ('{name}' , '{cat}' , {density} , '{GWP}','{ODP}','{POCP}','{EP}','{AP}');"""
        
        print(query)
        cursor.execute(query)


        

        

      
result = cursor.fetchall()
print('SQLite Version is {}'.format(result))



sqliteConnection.commit()
cursor.close()